In [341]:
import pandas as pd

# Load the 5 datasets
df1 = pd.read_csv('DBassign1/2016-Cities.csv')
df2 = pd.read_csv('DBassign1/2016-GHG_Emissions.csv')
df3 = pd.read_csv('DBassign1/2017-Community_Emissions.csv')
df4 = pd.read_csv('DBassign1/2017-Cities_Emission_Reduction_Targets.csv')
df5 = pd.read_csv('DBassign1/2023-Cities_Climate_Risk.csv')

# Print rows and columns for the 5 datasets
dataframes = [df1, df2, df3, df4, df5]
for i, df in enumerate(dataframes, start=1):
    print(f"DataFrame {i}: {df.shape[0]} rows, {df.shape[1]} columns")

DataFrame 1: 280 rows, 15 columns
DataFrame 2: 187 rows, 27 columns
DataFrame 3: 229 rows, 31 columns
DataFrame 4: 406 rows, 21 columns
DataFrame 5: 1370 rows, 20 columns


In [342]:
# variables for column names
org_no = 'Organization_id'
org_name = 'Organization_name'
c40 = 'C40_member'
GCoM = 'GCoM_member'
acc_year = 'accounting_year'
org_bound = 'Organization_boundary'

# to get city_id, need to know city
city_s = 'City Short Name'  # df2 and df1
city_n = 'City Name'    # df2
city = 'City' #df3, 4 and 5

## The final dataframe is called 'final_org_df' for the dataframe with the above NOTE: naming should be improved for the other tables

## Normalisation of data

### Org Table

####  Name normalisation

In [343]:
df1.rename(columns={'Account No': org_no,'Organisation': org_name}, inplace=True)
df2.rename(columns={'Account Number': org_no,'City Name': org_name}, inplace=True)
df3.rename(columns={'Account number': org_no,'Organization': org_name}, inplace=True)
df4.rename(columns={'Account No': org_no,'Organisation': org_name}, inplace=True)
df5.rename(columns={'Organization Number': org_no,'Organization Name': org_name}, inplace=True)

#### Combining org number and name into a new dataframe

In [344]:
# This takes all rows of all 5 datasets, but only adds the org name and number
combined_df = pd.concat([
    df1[[org_no, org_name]],
    df2[[org_no, org_name]],
    df3[[org_no, org_name]],
    df4[[org_no, org_name]],
    df5[[org_no, org_name]],
])

# Then all duplicates are dropped to get unique pairs
combined_df = combined_df.drop_duplicates().reset_index(drop=True)
print(combined_df.nunique())
combined_df

Organization_id      869
Organization_name    983
dtype: int64


,Organization_id,Organization_name
0,58796,Odder Kommune
1,36158,Comune di Napoli
2,62855,Egedal Municipality
3,61753,Yilan County
4,61790,"City of Emeryville, CA"
...,...,...
978,73671,Godoy Cruz
979,54306,Medan City Government
980,31178,City of Mumbai
981,863001,Puerto Varas


#### Removing duplicates from the combined df

In [345]:
# Group by 'Organization Number' and filter to find duplicates with different 'Organization Name'
duplicate_org_numbers = combined_df.groupby(org_no).filter(lambda x: x[org_name].nunique() > 1)

# To see each 'Organization Number' with its associated 'Organization Names'
duplicates_summary = duplicate_org_numbers.groupby(org_no)[org_name].unique()
print(duplicates_summary)


Organization_id
1093                [City of Atlanta, City of Atlanta, GA]
1184                  [City of Austin, City of Austin, TX]
1499          [Ajuntament de Barcelona, City of Barcelona]
3203                [City of Chicago, City of Chicago, IL]
3417                    [New York City, New York City, NY]
                               ...                        
60577    [Frederikshavn Kommune, Frederikshavn Municipa...
60588             [City of Alba-Iulia, City of Alba Iulia]
61753              [Yilan County, Yilan County Government]
63543      [Fredensborg Kommune, Fredensborg Municipality]
64014           [City of Cupertino, City of Cupertino, CA]
Name: Organization_name, Length: 114, dtype: object


In [346]:
# Function to return the row with the longest 'Organization Name' for each 'Organization Number'
def keep_longest_name(group):
    lengths = group[org_name].str.len()  # Get the length of each organization name
    return group.loc[lengths.idxmax()]  # Return the row with the longest organization name

# Apply the function to each group of duplicates
longest_names_df = duplicate_org_numbers.groupby(org_no).apply(keep_longest_name).reset_index(drop=True)

# print
print(longest_names_df)


     Organization_id           Organization_name
0               1093         City of Atlanta, GA
1               1184          City of Austin, TX
2               1499     Ajuntament de Barcelona
3               3203         City of Chicago, IL
4               3417           New York City, NY
..               ...                         ...
109            60577  Frederikshavn Municipality
110            60588          City of Alba-Iulia
111            61753     Yilan County Government
112            63543    Fredensborg Municipality
113            64014       City of Cupertino, CA

[114 rows x 2 columns]


In [347]:
# Step 1: Drop rows in newdf that have a matching 'Organization Number' in longest_names_df
org_numbers_to_remove = longest_names_df[org_no].unique()
newdf_filtered = combined_df[~combined_df[org_no].isin(org_numbers_to_remove)]

# Step 2: Concatenate newdf_filtered and longest_names_df
final_df = pd.concat([newdf_filtered, longest_names_df], ignore_index=True)
final_df

# final_df now contains the original data from newdf with duplicates removed, 
# and replaced by the rows with the longest 'Organization Name' for each 'Organization Number'


,Organization_id,Organization_name
0,58796,Odder Kommune
1,36158,Comune di Napoli
2,62855,Egedal Municipality
3,61790,"City of Emeryville, CA"
4,62180,Communauté urbaine du Grand Nancy
...,...,...
864,60577,Frederikshavn Municipality
865,60588,City of Alba-Iulia
866,61753,Yilan County Government
867,63543,Fredensborg Municipality


In [348]:
# To make sure that there are zero nulls in Organization Number and Name column
num_nulls_num = final_df[org_no].isnull().sum()
print(f"Number of nulls in {org_no}: {num_nulls_num}")
num_nulls_name = final_df[org_no].isnull().sum()
print(f"Number of nulls in {org_name}: {num_nulls_name}")

# To check if there are duplicates
print(f"Number of uniques in {org_no}: {final_df[org_no].nunique()}")
print(f"Number of uniques in {org_no}: {final_df[org_name].nunique()}")

# To save to a csv file, only needs to be done once, and afterwards, is commented out
# final_df.to_csv('DBassign1/combined_org_no_and_name.csv', index=False)

Number of nulls in Organization_id: 0
Number of nulls in Organization_name: 0
Number of uniques in Organization_id: 869
Number of uniques in Organization_id: 869


#### Normalisation of C40 

In [349]:
# Need to change df5 'C40 City'
df5['C40 City'].head(2)

0    False
1     True
Name: C40 City, dtype: bool

In [350]:
# Renaming the column
df5.rename(columns={'C40 City': c40}, inplace=True)
df5[c40].head(2)

0    False
1     True
Name: C40_member, dtype: bool

In [351]:
# Df1 to 4 all have the 'C40' column, and should only have the unique values 'C40' and null
dfs = [df1, df2, df3, df4]

# Iterate through each DataFrame and print unique values in the "C40" column
for i, df in enumerate(dfs, start=1):
    unique_values = df['C40'].unique()
    print(f"Unique values in C40 column of DataFrame {i}: {unique_values}")

Unique values in C40 column of DataFrame 1: [nan 'C40']
Unique values in C40 column of DataFrame 2: [nan 'C40']
Unique values in C40 column of DataFrame 3: [nan 'C40']
Unique values in C40 column of DataFrame 4: [nan 'C40']


In [352]:
# Assuming your DataFrames are named df1, df2, df3, df4
dataframes = [df1, df2, df3, df4]

# Iterate through each DataFrame and update the "C40" column
for df in dataframes:
    df['C40'] = df['C40'] == 'C40'

# At this point, each DataFrame will have the "C40" column with True where the value was 'C40', and False otherwise (including null values)

In [353]:
# Running this again to prove that it's only false and true now
# Iterate through each DataFrame and print unique values in the "C40" column
for i, df in enumerate(dfs, start=1):
    unique_values = df['C40'].unique()
    print(f"Unique values in C40 column of DataFrame {i}: {unique_values}")

Unique values in C40 column of DataFrame 1: [False  True]
Unique values in C40 column of DataFrame 2: [False  True]
Unique values in C40 column of DataFrame 3: [False  True]
Unique values in C40 column of DataFrame 4: [False  True]


In [354]:
df1.rename(columns={'C40': c40}, inplace=True)
df2.rename(columns={'C40': c40}, inplace=True)
df3.rename(columns={'C40': c40}, inplace=True)
df4.rename(columns={'C40': c40}, inplace=True)

df3[c40].head(2)

0    False
1     True
Name: C40_member, dtype: bool

In [355]:
# Now we need to add the C40 column to the 'final_df'
# So we need to check if ther are inconsistencies in the C40 value between datasets (say it might be false in 2017 reduction target but true in 
# 2023 climate risk for the same org number)


# Assuming the 'org id' column and 'C40' column are named 'org id' and 'C40' respectively in each df
dataframes = [df1, df2, df3, df4, df5]

# Combine all 'org id' and 'C40' into one DataFrame
combined_c40 = pd.concat([df[[org_no, c40]] for df in dataframes], ignore_index=True)

# Check if there are inconsistencies in 'C40' values for the same 'org id'
inconsistent_c40 = combined_c40.groupby(org_no).filter(lambda x: x[c40].nunique() > 1)

if not inconsistent_c40.empty:
    print("Warning: Inconsistencies found in 'C40' values for the same 'org id'.")
    print(inconsistent_c40)
else:
    print("No inconsistencies in 'C40' values found.")

      Organization_id  C40_member
22              36159       False
74              31151        True
75              31151        True
92              36254        True
93              36254        True
...               ...         ...
2223            35885        True
2226            31151       False
2312            35870        True
2318            36254       False
2376            35885        True

[61 rows x 2 columns]


In [356]:
# Find the unique org numbers
inconsistent_c40[org_no].unique()

array([36159, 31151, 36254, 35894, 31186, 35874, 35885, 35870],
      dtype=int64)

In [357]:
# 
unique_inconsistent_c40 = df5.loc[df5[org_no].isin([36159, 31151, 36254, 35894, 31186, 35874, 35885, 35870])]
# Select the 'org no' and 'C40' columns, then drop duplicates to get unique pairs
unique_pairs = unique_inconsistent_c40[[org_no, c40]].drop_duplicates()

# Display the unique pairs
print(unique_pairs)


      Organization_id  C40_member
93              35894        True
111             31186       False
135             35885        True
167             35870        True
421             35874        True
477             36159        True
636             31151       False
1216            36254       False


In [358]:
# Assuming 'org_no' is the column name in both combined_c40 and unique_pairs DataFrames
# Get a list of org_no values to remove from combined_c40
org_no_to_remove = unique_pairs[org_no].unique()

# Filter out rows from combined_c40 where org_no matches any value in org_no_to_remove
combined_c40_filtered = combined_c40[~combined_c40[org_no].isin(org_no_to_remove)]

# combined_c40_filtered now contains rows from combined_c40 excluding those with org_no in unique_pairs
unique_pairs_combined = combined_c40_filtered[[org_no, c40]].drop_duplicates()

# Display the unique pairs
print(unique_pairs_combined)


      Organization_id  C40_member
0               58796       False
1               36158       False
2               62855       False
3               61753       False
4               61790       False
...               ...         ...
2454            73671       False
2457            54306       False
2461            31178        True
2466           863001       False
2471           924874       False

[861 rows x 2 columns]


In [359]:
# Combine the 2 unique pairs dfs into 1
combined_unique_pairs = pd.concat([unique_pairs_combined, unique_pairs]).drop_duplicates(subset=[org_no])
combined_unique_pairs

,Organization_id,C40_member
0,58796,False
1,36158,False
2,62855,False
3,61753,False
4,61790,False
...,...,...
167,35870,True
421,35874,True
477,36159,True
636,31151,False


In [360]:
final_df_with_c40 = pd.merge(final_df, combined_unique_pairs, on=org_no, how='left')
final_df_with_c40

,Organization_id,Organization_name,C40_member
0,58796,Odder Kommune,False
1,36158,Comune di Napoli,False
2,62855,Egedal Municipality,False
3,61790,"City of Emeryville, CA",False
4,62180,Communauté urbaine du Grand Nancy,False
...,...,...,...
864,60577,Frederikshavn Municipality,False
865,60588,City of Alba-Iulia,False
866,61753,Yilan County Government,False
867,63543,Fredensborg Municipality,False


In [361]:
final_df_with_c40[c40].unique()
final_df_with_c40_falseTrue = df5.loc[df5[org_no].isin([58796, 36158, 62855, 61790, 62180, 60577, 60588, 61753, 63543, 64014])]
print(final_df_with_c40_falseTrue[[org_no, c40]])

      Organization_id  C40_member
296             64014       False
673             64014       False
675             60577       False
873             60588       False
985             63543       False
1114            60577       False


#### Normalisation of GCoM

In [362]:
# Need to change df5 'C40 City'
df5['GCoM City'].head(3)

0     True
1    False
2    False
Name: GCoM City, dtype: bool

In [363]:
# Renaming the column
df5.rename(columns={'GCoM City': GCoM}, inplace=True)
df5[GCoM].head(2)

0     True
1    False
Name: GCoM_member, dtype: bool

In [364]:
# Since only df5 has GCoM, we only need to get unique pairs between org_no and GCoM for df5
unique_pairs_gcom = df5[[org_no, GCoM]].drop_duplicates()
unique_pairs_gcom


,Organization_id,GCoM_member
0,840926,True
1,51075,False
2,863190,False
3,930366,True
4,60236,True
...,...,...
1356,31187,True
1357,44191,True
1359,31178,True
1364,863001,False


In [365]:
# Now we merge the unique_pairs_gcom into our final_df(_with_c40)
final_df_with_gcom = pd.merge(final_df_with_c40, unique_pairs_gcom, on=org_no, how='left')
final_df_with_gcom

,Organization_id,Organization_name,C40_member,GCoM_member
0,58796,Odder Kommune,False,NaN
1,36158,Comune di Napoli,False,NaN
2,62855,Egedal Municipality,False,NaN
3,61790,"City of Emeryville, CA",False,NaN
4,62180,Communauté urbaine du Grand Nancy,False,NaN
...,...,...,...,...
864,60577,Frederikshavn Municipality,False,True
865,60588,City of Alba-Iulia,False,True
866,61753,Yilan County Government,False,NaN
867,63543,Fredensborg Municipality,False,True


In [366]:
# We don't change the NaN values in GCoM_member to false, because they might be 'True' in reality, so we keep them as NaN meaning don't know

#### Normalisation of Accounting year

In [367]:
# Renaming the column, it only exists in df3 - 2017 community emmissions
df3.rename(columns={'Accounting year': acc_year}, inplace=True)
df3[acc_year].head(3)

0    2013-07-01 - 2014-06-30
1    2014-01-01 - 2014-12-31
2    2015-01-01 - 2015-12-31
Name: accounting_year, dtype: object

In [368]:
# Since only df3 has accounting year, we only need to get unique pairs between org_no and acc_year for df3
unique_pairs_accyear = df3[[org_no, acc_year]].drop_duplicates()
unique_pairs_accyear

,Organization_id,accounting_year
0,49363,2013-07-01 - 2014-06-30
1,31171,2014-01-01 - 2014-12-31
2,3417,2015-01-01 - 2015-12-31
3,59537,2015-01-01 - 2015-12-31
4,35894,2013-01-01 - 2013-12-31
...,...,...
224,19233,2014-01-01 - 2014-12-31
225,60273,2014-01-01 - 2014-12-31
226,31148,2015-12-01 - 2016-12-31
227,54119,2016-01-01 - 2016-12-31


In [369]:
# Now we merge the unique_pairs_accyear into our final_df(_with_gcom)
final_df_with_accyear = pd.merge(final_df_with_gcom, unique_pairs_accyear, on=org_no, how='left')
final_df_with_accyear

# We keep the NaN for accounting year, although there should be quite a lot
# since only at most 229 rows has values in accounting year

,Organization_id,Organization_name,C40_member,GCoM_member,accounting_year
0,58796,Odder Kommune,False,NaN,NaN
1,36158,Comune di Napoli,False,NaN,NaN
2,62855,Egedal Municipality,False,NaN,2014-01-01 - 2014-12-31
3,61790,"City of Emeryville, CA",False,NaN,2014-01-01 - 2014-12-31
4,62180,Communauté urbaine du Grand Nancy,False,NaN,2016-01-01 - 2016-12-31
...,...,...,...,...,...
864,60577,Frederikshavn Municipality,False,True,NaN
865,60588,City of Alba-Iulia,False,True,NaN
866,61753,Yilan County Government,False,NaN,2014-01-01 - 2014-12-31
867,63543,Fredensborg Municipality,False,True,NaN


#### Normalisation of Org Boundary

In [370]:
# Renaming the column, it only exists in df2 and df3
df2.rename(columns={'Boundary': org_bound}, inplace=True)
print(df2[org_bound].head(3))

df3.rename(columns={'Boundary': org_bound}, inplace=True)
print(df3[org_bound].head(3))

0    Other: The regional entity that constitutes th...
1                                  A metropolitan area
2        Administrative boundary of a local government
Name: Organization_boundary, dtype: object
0                              A metropolitan area
1    Administrative boundary of a local government
2    Administrative boundary of a local government
Name: Organization_boundary, dtype: object


In [371]:
# This takes all rows of all 5 datasets, but only adds the org name and number
unique_pairs_orgbound = pd.concat([
    df2[[org_no, org_bound]],
    df3[[org_no, org_bound]],
])

# Then all duplicates are dropped to get unique pairs
unique_pairs_orgbound = unique_pairs_orgbound.drop_duplicates().reset_index(drop=True)

# Calculate the number of NaN values in each column of the DataFrame
nan_counts = unique_pairs_orgbound.isna().sum()
# Print the number of NaN values for each column
print(nan_counts)

# print
print(unique_pairs_orgbound.nunique()) # to see how many unique there are in org bound
unique_pairs_orgbound

Organization_id          0
Organization_boundary    0
dtype: int64
Organization_id          263
Organization_boundary     17
dtype: int64


,Organization_id,Organization_boundary
0,35894,Other: The regional entity that constitutes th...
1,35898,A metropolitan area
2,54128,Administrative boundary of a local government
3,35879,Administrative boundary of a local government
4,50558,Administrative boundary of a local government
...,...,...
271,43911,Administrative boundary of a local government
272,60374,Administrative boundary of a local government
273,63999,Administrative boundary of a local government
274,50371,Administrative boundary of a local government


In [372]:
# Now we merge the unique_pairs_orgbound into our final_df(_with_accyear)
final_org_df = pd.merge(final_df_with_accyear, unique_pairs_orgbound, on=org_no, how='left')
final_org_df

,Organization_id,Organization_name,C40_member,GCoM_member,accounting_year,Organization_boundary
0,58796,Odder Kommune,False,NaN,NaN,NaN
1,36158,Comune di Napoli,False,NaN,NaN,NaN
2,62855,Egedal Municipality,False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government
3,61790,"City of Emeryville, CA",False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government
4,62180,Communauté urbaine du Grand Nancy,False,NaN,2016-01-01 - 2016-12-31,A metropolitan area
...,...,...,...,...,...,...
877,60577,Frederikshavn Municipality,False,True,NaN,NaN
878,60588,City of Alba-Iulia,False,True,NaN,NaN
879,61753,Yilan County Government,False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government
880,63543,Fredensborg Municipality,False,True,NaN,NaN


#### Removing duplicates introduced from org boundary normalisation

In [373]:
org_bound_dupes = final_org_df[final_org_df['Organization_id'].duplicated()]

print(f"org_bound_dupes has {org_bound_dupes.shape[0]} rows.")
org_bound_dupes

# Below to see 59996, which does have 2 rows with different org boundaries
# final_org_df[final_org_df['Organization_id'] == 59996]

org_bound_dupes has 13 rows.


,Organization_id,Organization_name,C40_member,GCoM_member,accounting_year,Organization_boundary
46,3422,Greater London Authority,True,True,2014-01-01 - 2014-12-31,Other: Admistrative boundary of the Greater Lo...
71,59595,"City of Brisbane, CA",False,NaN,2015-05-01 - 2016-05-01,A metropolitan area
97,59996,Batangas City,False,NaN,2015-01-01 - 2015-12-31,Administrative boundary of a local government
99,31170,Metropolitan Municipality of Lima,True,True,2015-01-01 - 2015-12-31,Administrative boundary of a local government
111,60656,"City of Piedmont, CA",False,True,2010-01-01 - 2011-01-01,Other
121,31180,Región Metropolitana de Santiago,True,True,2013-01-01 - 2013-12-31,Administrative boundary of a local government
132,31154,Bogotá Distrito Capital,True,True,2017-01-01 - 2017-12-31,A metropolitan area
770,13067,"City of New Orleans, LA",True,True,2014-01-01 - 2014-12-31,Administrative boundary of a local government
783,31151,City of Basel,False,True,2016-01-01 - 2016-12-31,Administrative boundary of a local government
800,35863,eThekwini Municipality,True,True,2015-01-01 - 2015-12-31,Administrative boundary of a local government


In [374]:
# checking if the all of the above are in df3 (it's 2017, so newer)
org_bound_dupes = org_bound_dupes[[org_no, org_bound]]
df3_org = df3[[org_no, org_bound]]

df3_org = df3_org[df3_org[org_no].isin(org_bound_dupes[org_no])]

print(f"df3_org has {df3_org.shape[0]} rows.")
df3_org

df3_org has 13 rows.


,Organization_id,Organization_boundary
4,35894,A metropolitan area
18,59996,Administrative boundary of a local government
20,60656,Other
40,31154,A metropolitan area
47,59595,A metropolitan area
56,31180,Administrative boundary of a local government
67,49360,Administrative boundary of a local government
146,31170,Administrative boundary of a local government
147,3422,Other: Admistrative boundary of the Greater Lo...
156,31151,Administrative boundary of a local government


In [375]:
# Filter final_org_df for rows where org_no is in df3_org
matching_org_no = df3_org[org_no].unique()
org_bound_dupes_df = final_org_df[final_org_df[org_no].isin(matching_org_no)]

# Filter org_bound_dupes_df based on matching org_bound in df3_org
new_df_cleaned = org_bound_dupes_df.merge(df3_org[[org_no, org_bound]], on=[org_no, org_bound], how='inner')
new_df_cleaned

# Remove rows from final_org_df
final_org_df = final_org_df[~final_org_df[org_no].isin(matching_org_no)]

# Concatenate the reduced final_org_df with the cleaned new_df
final_cleaned_df = pd.concat([final_org_df, new_df_cleaned], ignore_index=True)

final_cleaned_df


,Organization_id,Organization_name,C40_member,GCoM_member,accounting_year,Organization_boundary
0,58796,Odder Kommune,False,NaN,NaN,NaN
1,36158,Comune di Napoli,False,NaN,NaN,NaN
2,62855,Egedal Municipality,False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government
3,61790,"City of Emeryville, CA",False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government
4,62180,Communauté urbaine du Grand Nancy,False,NaN,2016-01-01 - 2016-12-31,A metropolitan area
...,...,...,...,...,...,...
864,31151,City of Basel,False,True,2016-01-01 - 2016-12-31,Administrative boundary of a local government
865,35863,eThekwini Municipality,True,True,2015-01-01 - 2015-12-31,Administrative boundary of a local government
866,35894,"Ville de Montreal, QC",True,True,2013-01-01 - 2013-12-31,A metropolitan area
867,49360,Pretoria - Tshwane,True,True,2014-07-01 - 2015-06-30,Administrative boundary of a local government


## Getting city id

#### Importing city table

In [376]:
# DF for cities
df_cities = pd.read_csv('DBassign1/cities_table.csv')

print(df_cities.nunique())
df_cities

Unnamed: 0          789
city_location       531
average_altitude    163
average_temp        123
city_name           789
country_id           89
city_id             789
dtype: int64


,Unnamed: 0,city_location,average_altitude,average_temp,city_name,country_id,city_id
0,0,"(56.168393, 10.137373)",NaN,NaN,Aarhus,24,1
1,1,"(56.168393, 10.137373)",NaN,NaN,Aarhus Kommune,24,2
2,2,"(31.323126, 34.344025)",78.0,25.0,Abasan Al-Kabira,77,3
3,3,POINT (-4.0083 5.36),NaN,NaN,Abidjan,21,4
4,4,"(40.12408, -75.119511)",86.0,13.6,Abington,84,5
...,...,...,...,...,...,...,...
784,784,NaN,NaN,NaN,la Ciudad de Paraná,2,785
785,785,POINT (-8.44582 40.5753),NaN,NaN,Águeda,66,786
786,786,NaN,NaN,NaN,Åseda,78,787
787,787,"(54.891456, 10.404684)",10.0,10.0,Ærøskøbing,24,788


In [377]:
# df1.rename(columns={'Account No': org_no,'Organisation': org_name}, inplace=True)
# df2.rename(columns={'Account Number': org_no,'City Name': city_n}, inplace=True)
# df3.rename(columns={'Account number': org_no,'Organization': org_name}, inplace=True)
# df4.rename(columns={'Account No': org_no,'Organisation': org_name}, inplace=True)
# df5.rename(columns={'Organization Number': org_no,'Organization Name': org_name}, inplace=True)

# city_n

# a new city df of all df1 to 5 combined with the city names
city_df = pd.concat([
    df1[[org_no, org_name, city_s]],
    df2[[org_no, org_name, city_s]],
    df3[[org_no, org_name, city]],
    df4[[org_no, org_name, city]],
    df5[[org_no, org_name, city]],
])

print(f"city_df has {city_df.shape[0]} rows.")
print(city_df.nunique())
city_df.head(3)

city_df has 2472 rows.
Organization_id      869
Organization_name    983
City Short Name      232
City                 589
dtype: int64


,Organization_id,Organization_name,City Short Name,City
0,58796,Odder Kommune,Odder Kommune,NaN
1,36158,Comune di Napoli,Napoli,NaN
2,62855,Egedal Municipality,Egedal Municipality,NaN


In [378]:
data = []

for index, row in df_cities.iterrows():
    city_name = row['city_name']
    city_id = row['city_id']
    
    matches = city_df[(city_df['Organization_name'] == city_name) | 
                      (city_df['City Short Name'] == city_name) | 
                      (city_df['City'] == city_name)]
    
    for _, match_row in matches.iterrows():
        data.append({'Organization_id': match_row['Organization_id'], 'city_id': city_id, 'city_name': city_name})

# Convert the list of dictionaries to a DataFrame
org_no_and_city_id = pd.DataFrame(data)

print(org_no_and_city_id.nunique())
org_no_and_city_id

Organization_id    582
city_id            789
city_name          789
dtype: int64


,Organization_id,city_id,city_name
0,54408,1,Aarhus
1,54408,1,Aarhus
2,54408,1,Aarhus
3,54408,2,Aarhus Kommune
4,54408,2,Aarhus Kommune
...,...,...,...
2886,55325,786,Águeda
2887,859254,787,Åseda
2888,58609,788,Ærøskøbing
2889,58609,788,Ærøskøbing


In [379]:
dupes_org_no_and_city_id = org_no_and_city_id[org_no_and_city_id.duplicated()]
dupes_org_no_and_city_id

,Organization_id,city_id,city_name
1,54408,1,Aarhus
2,54408,1,Aarhus
4,54408,2,Aarhus Kommune
5,54408,2,Aarhus Kommune
6,54408,2,Aarhus Kommune
...,...,...,...
2883,55325,786,Águeda
2884,55325,786,Águeda
2885,55325,786,Águeda
2886,55325,786,Águeda


In [380]:
dupes_org_no_and_city_id[dupes_org_no_and_city_id[org_no] == 55325]

,Organization_id,city_id,city_name
2879,55325,786,Águeda
2880,55325,786,Águeda
2881,55325,786,Águeda
2882,55325,786,Águeda
2883,55325,786,Águeda
2884,55325,786,Águeda
2885,55325,786,Águeda
2886,55325,786,Águeda


In [381]:
print(df5['Organization_id'].isna().sum())
print(df5['Organization_name'].isna().sum())
print(df5['City'].isna().sum())
df5.nunique()

0
0
439


Questionnaire                                                          1
Organization_id                                                      741
Organization_name                                                    741
City                                                                 443
Country/Area                                                          79
CDP Region                                                            10
C40_member                                                             2
GCoM_member                                                            2
Access                                                                 1
Assessment attachment and/or direct link                            1299
Confirm attachment/link provided                                      38
Boundary of assessment relative to jurisdiction boundary             341
Year of publication or approval                                       24
Factors considered in assessment                   

#### Dropping duplicates

In [382]:
# Ensuring new_df contains only unique pairs of Organization_id and city_id
org_no_and_city_id_unique = org_no_and_city_id.drop_duplicates(subset=['Organization_id', 'city_id'])

print(org_no_and_city_id_unique.nunique())
org_no_and_city_id_unique

Organization_id    582
city_id            789
city_name          789
dtype: int64


,Organization_id,city_id,city_name
0,54408,1,Aarhus
3,54408,2,Aarhus Kommune
7,63616,3,Abasan Al-Kabira
11,36004,4,Abidjan
12,73413,4,Abidjan
...,...,...,...
2877,54620,785,la Ciudad de Paraná
2878,55325,786,Águeda
2887,859254,787,Åseda
2888,58609,788,Ærøskøbing


In [383]:
# Getting all dupes
dupes_org_id_with_both = org_no_and_city_id_unique[org_no_and_city_id_unique['Organization_id'].duplicated(keep=False)]

print(dupes_org_id_with_both.nunique())
dupes_org_id_with_both

Organization_id    198
city_id            417
city_name          417
dtype: int64


,Organization_id,city_id,city_name
0,54408,1,Aarhus
3,54408,2,Aarhus Kommune
13,58485,5,Abington
15,58485,6,Abington Township
19,31146,8,Addis Ababa
...,...,...,...
2860,52893,780,Yonkers
2865,46473,783,Zaragoza
2870,35449,784,Zürich
2878,55325,786,Águeda


In [384]:
dupes_org_id_with_both = dupes_org_id_with_both.sort_values(by='Organization_id', ascending=True)

# dupes_org_id_with_both.to_csv('DBassign1/city_id_dupes.csv', sep=';', index=False)

In [385]:
# now removing those from org_no_and_city_id_unique that appears in dupes_org_id_with_both

# Get the Organization_id values to exclude
org_ids_to_exclude = dupes_org_id_with_both['Organization_id'].unique()

# Filter org_no_and_city_id_unique to exclude those Organization_id values
filtered_org_no_and_city_id_df = org_no_and_city_id_unique[~org_no_and_city_id_unique['Organization_id'].isin(org_ids_to_exclude)]

filtered_org_no_and_city_id_df



,Organization_id,city_id,city_name
7,63616,3,Abasan Al-Kabira
11,36004,4,Abidjan
12,73413,4,Abidjan
17,36039,7,Accra
33,46263,11,Ahmedabad
...,...,...,...
2862,58613,781,York
2864,43921,782,Zagreb
2877,54620,785,la Ciudad de Paraná
2887,859254,787,Åseda


#### Importing cities_no_dupes that has filtered out dupes in cities_table with dupes_org_id_with_both

In [386]:
# DF for cities
df_cities_no_dupes = pd.read_csv('DBassign1/cities_no_dupes.csv')

print(df_cities_no_dupes.nunique())
df_cities_no_dupes

## 198 unique org id, which is also the case for dupes_org_id_with_both

Unnamed: 0          199
city_location       197
average_altitude    119
average_temp         97
city_name           199
country_id           47
city_id             199
Organization_id     198
dtype: int64


,Unnamed: 0,city_location,average_altitude,average_temp,city_name,country_id,city_id,Organization_id
0,0,"(33.7489954, -84.3879824)",272.0,16.9,City of Atlanta,84,138,1093
1,1,"(30.2672, -97.7431)",149.0,20.0,City of Austin,84,139,1184
2,2,"(52.480174, -1.902907)",140.0,9.2,Birmingham City Council,87,66,1850
3,3,"(44.4758825, -73.212072)",61.0,7.7,City of Burlington,84,148,2430
4,4,"(41.8781136, -87.6297982)",181.0,9.4,City of Chicago,84,152,3203
...,...,...,...,...,...,...,...,...
194,194,"(48.692054, 6.184417)",212.0,10.5,Communauté urbaine du Grand Nancy,31,239,62180
195,195,"(55.795045, 12.238733)",29.0,8.0,Egedal,24,283,62855
196,196,"(40.037875, -76.305514)",112.0,11.5,"City of Lancaster, PA",84,181,62864
197,197,"_\n(25.790654, -80.130045)",NaN,23.9,"Miami Beach, FL",84,470,63999


In [387]:
# now, with df_cities_no_dupes and filtered_org_no_and_city_id_df
# just need to take org_no from my org_table, look it up in the 2 dfs, and then add city_id
combined_city_org = pd.concat([
    df_cities_no_dupes[[org_no, 'city_id']],
    filtered_org_no_and_city_id_df[[org_no, 'city_id']],
])

combined_city_org

,Organization_id,city_id
0,1093,138
1,1184,139
2,1850,66
3,2430,148
4,3203,152
...,...,...
2862,58613,781
2864,43921,782
2877,54620,785
2887,859254,787


In [388]:
# Perform a left merge to keep all rows from final_cleaned_df and only add matching 'city_id' from combined_city_org
final_cleaned_df_with_city_id = pd.merge(final_cleaned_df, 
                                         combined_city_org[['Organization_id', 'city_id']], 
                                         on='Organization_id', 
                                         how='left')

print(final_cleaned_df_with_city_id.nunique())
print(f"-----amount of nulls/NaN in city_id {final_cleaned_df_with_city_id['city_id'].isna().sum()}")
final_cleaned_df_with_city_id

Organization_id          869
Organization_name        869
C40_member                 2
GCoM_member                2
accounting_year           34
Organization_boundary     14
city_id                  577
dtype: int64
-----amount of nulls/NaN in city_id 287


,Organization_id,Organization_name,C40_member,GCoM_member,accounting_year,Organization_boundary,city_id
0,58796,Odder Kommune,False,NaN,NaN,NaN,524.0
1,36158,Comune di Napoli,False,NaN,NaN,NaN,504.0
2,62855,Egedal Municipality,False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government,283.0
3,61790,"City of Emeryville, CA",False,NaN,2014-01-01 - 2014-12-31,Administrative boundary of a local government,163.0
4,62180,Communauté urbaine du Grand Nancy,False,NaN,2016-01-01 - 2016-12-31,A metropolitan area,239.0
...,...,...,...,...,...,...,...
865,31151,City of Basel,False,True,2016-01-01 - 2016-12-31,Administrative boundary of a local government,53.0
866,35863,eThekwini Municipality,True,True,2015-01-01 - 2015-12-31,Administrative boundary of a local government,160.0
867,35894,"Ville de Montreal, QC",True,True,2013-01-01 - 2013-12-31,A metropolitan area,747.0
868,49360,Pretoria - Tshwane,True,True,2014-07-01 - 2015-06-30,Administrative boundary of a local government,576.0


In [389]:
# final_cleaned_df_with_city_id
final_cleaned_df_with_city_id.to_csv('DBassign1/org_table.csv', sep=';', index=False)

# Ignore the below, old stuff that should be deleted

In [390]:
# intentional code that fails to stop run all to run below
final_cleaned_df_with_city_id.to_csv('DBassign1/or)

SyntaxError: unterminated string literal (detected at line 2) (687147406.py, line 2)

In [ ]:
# a new city df with the city names
city_df = pd.concat([
    df1[[org_no, org_name, city_s]],
    df2[[org_no, org_name, city_s]],
    df3[[org_no, org_name, city]],
    df4[[org_no, org_name, city]],
    df5[[org_no, org_name, city]],
])

# Then all duplicates are dropped to get unique pairs
city_df = city_df.drop_duplicates().reset_index(drop=True)
print(city_df.nunique())
city_df

#### Checking for NaN and duplicates

In [ ]:
# check if there are rows with NaN in both City Short Name and City
rows_with_nan = city_df[city_df['City Short Name'].isna() & city_df['City'].isna()]

# Display the filtered rows
rows_with_nan

In [ ]:
# create new column City Name with Organization_name as the values
rows_with_nan_updated = rows_with_nan.copy()  # Create a copy to modify
rows_with_nan_updated['City Name'] = rows_with_nan_updated['Organization_name']

# remove City Short Name and City
rows_with_nan_updated = rows_with_nan_updated.drop(columns=[city_s, 'City'])
rows_with_nan_updated

In [ ]:
# Filter to exclude rows where both 'City Short Name' and 'City' are NaN
city_without_nan = city_df.dropna(subset=['City Short Name', 'City'], how='all')
city_without_nan

In [ ]:
# 1. Pairs with a value in 'City Short Name' but NaN in 'City'
condition_1 = city_without_nan['City Short Name'].notna() & city_without_nan['City'].isna()
count_condition_1 = condition_1.sum()

# 2. Pairs with a value in 'City' but NaN in 'City Short Name'
condition_2 = city_without_nan['City'].notna() & city_without_nan['City Short Name'].isna()
count_condition_2 = condition_2.sum()

# 3. Pairs that do not belong to the above categories (both non-NaN values)
# Since city_without_nan was filtered to exclude rows where both are NaN, 
# this condition implicitly includes rows where both columns have non-NaN values
condition_3 = ~(condition_1 | condition_2)
count_condition_3 = condition_3.sum()

# Print the counts
print(f"Number of rows with value in 'City Short Name' but NaN in 'City': {count_condition_1}")
print(f"Number of rows with value in 'City' but NaN in 'City Short Name': {count_condition_2}")
print(f"Number of rows not belonging to the above categories: {count_condition_3}")


In [ ]:
new_df = city_without_nan.copy()
new_df['City Name'] = new_df.apply(
    lambda row: row['City Short Name'] if pd.isna(row['City']) else row['City'], axis=1
)

# Drop 'City Short Name' and 'City' columns
new_df.drop(columns=['City Short Name', 'City'], inplace=True)
new_df

In [ ]:
final_city_df = pd.concat([rows_with_nan_updated, new_df])

print(final_city_df.nunique())
final_city_df

In [ ]:
# Ensuring new_df contains only unique pairs of Organization_id and city_id
org_name_duplicates = final_city_df.duplicated(subset=['Organization_id', 'Organization_name'])
city_name_duplicates = final_city_df.duplicated(subset=['Organization_id', 'City Name'])

#### Identifying and removing duplicate pairs

In [ ]:
# Identifying duplicate pairs based on 'Organization_id' and 'City Name'
duplicate_pairs = final_city_df.duplicated(subset=['Organization_id', 'City Name'], keep=False)

print(f"Are there any duplicate pairs? {duplicate_pairs.any()}")

# rows display
print("Duplicate pairs based on Organization_id and City Name:")
print(final_city_df[duplicate_pairs])


In [ ]:
# Identifying duplicate pairs based on 'Organization_id' and 'Organization_name'
duplicate_pairs = final_city_df.duplicated(subset=['Organization_id', 'Organization_name'], keep=False)

# rows display
print("Duplicate pairs based on Organization_id and City Name:")
final_city_df[duplicate_pairs]


In [ ]:
# Dropping duplicate pairs based on 'Organization_id' and 'City Name', keeping the first occurrence
final_city_df = final_city_df.drop_duplicates(subset=['Organization_id', 'City Name'], keep='first')
final_city_df

In [ ]:
# Dropping duplicate pairs based on 'Organization_id' and 'City Name', keeping the longest value in City Name
# by sorting first
final_city_df = final_city_df.sort_values(by=['City Name'], 
                                   key=lambda x: x.str.len(), ascending=False)

# Step 2: Drop duplicates, keeping the first occurrence (which has the longest 'City Name')
final_city_df = final_city_df.drop_duplicates(subset=['Organization_id'], keep='first')

# Optional: Reset index if desired
final_city_df.reset_index(drop=True, inplace=True)
final_city_df


In [ ]:
final_city_df = final_city_df.sort_values(by='City Name', ascending=True)


pd.set_option('display.max_rows', None)

print(final_city_df)

pd.reset_option('display.max_rows')

In [ ]:
print(final_city_df.nunique())
final_city_df
# final_city_df.to_csv('DBassign1/city_temp.csv', sep=';', index=False)

#### Find duplicates

In [ ]:
import numpy as np
final_org_df['city_id'] = np.nan
final_org_df

In [ ]:
# Group by 'Organization Number' and filter to find duplicates with different 'Organization Name'
duplicate_org_numbers = combined_df.groupby(org_no).filter(lambda x: x[org_name].nunique() > 1)

# To see each 'Organization Number' with its associated 'Organization Names'
duplicates_summary = duplicate_org_numbers.groupby(org_no)[org_name].unique()
print(duplicates_summary)


#### Saving to csv

In [ ]:
final_org_df = final_org_df.drop(columns=[city_s, 'City'])

print(final_org_df.nunique())

final_org_df

In [ ]:
# make a mapping
city_mapping = final_city_df[['Organization_id', 'City Name']].set_index('Organization_id')['City Name']

# Map 'Organization_id' in final_org_df to 'City Name' using the mapping
# The 'map' function replaces each 'Organization_id' with its corresponding 'City Name'
final_org_df['City Name'] = final_org_df['Organization_id'].map(city_mapping)

final_org_df

In [ ]:
# final_org_df
# final_org_df.to_csv('DBassign1/org_table.csv', sep=';', index=False)